# Importing Librarires

In [4]:
import pandas as pd
import os
import shutil

## Arragnging all the images in 7 sub folders under a Universal folder Reorganized.

In [6]:
data_dir=os.getcwd()+"/Dataset/all_images"
dest_dir=os.getcwd()+"/Dataset/Reorganized/"
skin_df=pd.read_csv('Dataset/HAM10000_metadata.csv')
print(skin_df['dx'].value_counts())
label=skin_df['dx'].unique().tolist()
label_images=[]
for i in label:
    os.mkdir(dest_dir + str(i) + "/")
    sample = skin_df[skin_df['dx'] == i]['image_id']
    label_images.extend(sample)
    for id in label_images:
        shutil.copyfile((data_dir + "/"+ id +".jpg"), (dest_dir + i + "/"+id+".jpg"))
    label_images=[]    

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64
